In [52]:
import pandas as pd
import json

from fuzzywuzzy import fuzz

### Para este codigo, debemos haber ejecutado anteriormente *01_restaurantes* y *categorias*, ya que trabajaremos con los archivos obtenidos

### 01_restaurantes

In [53]:
# Especifica la ruta del archivo JSON
ruta_archivo = 'restaurantes_A.json'
# Lee el archivo JSON y carga los datos
with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
    # Carga el archivo JSON
    objetos_json = [json.loads(line) for line in archivo]
# Convierte los datos en un DataFrame
df_restaurantes = pd.DataFrame(objetos_json)

# Eliminamos las columnas que no tendremos en cuenta para obtener las categorias
df_restaurantes = df_restaurantes.drop(['latitud', 'longitud', 'avg_rating', 'reviews_count', 'servicios', 'estado', 'direccion', 'ciudad'], axis=1)

In [54]:
df_restaurantes.head(5)

,index,nombre,identificador_google,categorias,identificador_yelp
0,0,Cape Seafood Shack,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,[Restaurant],None
1,1,Fresh Point Country Buffet,0x88c2e4e34f1ed783:0x76c5da381c499d79,[Buffet restaurant],None
2,2,Oneyda's Bakery,0x88dae191ee505917:0x6ba3e25388d3fad4,"[Bakery, Deli]",None
3,3,Annie's Bake Shoppe,0x88d9c754413f6c9d:0x1f93eff5e0ba9c16,"[Bakery, Gift basket store]",None
4,4,Tropical Park Liquors,0x88d9b99475d9fd7b:0xea6083d207b2471a,[Liquor store],None


In [55]:
# Hay categorias agrupadas en listas y otras que solo estan separadas por coma. Las que estan separas por coma, se transforman en listas
df_restaurantes['categorias'] = df_restaurantes['categorias'].apply(lambda x: x if isinstance(x, list) else [c.strip() for c in x.split(',')])
# Divide las listas en filas
df_restaurantes = df_restaurantes.explode('categorias')
# A las categorias obtenidas, las pasamos a minusculas y eliminamos posibles espacios en blanco
df_restaurantes['categorias'] = df_restaurantes['categorias'].apply(lambda x: x.lower().lstrip())


In [56]:
df_restaurantes

,index,nombre,identificador_google,categorias,identificador_yelp
0,0,Cape Seafood Shack,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,restaurant,None
1,1,Fresh Point Country Buffet,0x88c2e4e34f1ed783:0x76c5da381c499d79,buffet restaurant,None
2,2,Oneyda's Bakery,0x88dae191ee505917:0x6ba3e25388d3fad4,bakery,None
2,2,Oneyda's Bakery,0x88dae191ee505917:0x6ba3e25388d3fad4,deli,None
3,3,Annie's Bake Shoppe,0x88d9c754413f6c9d:0x1f93eff5e0ba9c16,bakery,None
...,...,...,...,...,...
26435,26610,Colony Grill - St. Petersburg,None,beer bar,esBGrrmuZzSiECyRBoKvvA
26435,26610,Colony Grill - St. Petersburg,None,nightlife,esBGrrmuZzSiECyRBoKvvA
26435,26610,Colony Grill - St. Petersburg,None,wine bars,esBGrrmuZzSiECyRBoKvvA
26435,26610,Colony Grill - St. Petersburg,None,pizza,esBGrrmuZzSiECyRBoKvvA


### categorias

In [57]:
# Especifica la ruta del archivo JSON
ruta_archivo = 'categorias_A.json'
# Lee el archivo JSON y carga los datos
with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
    # Carga el archivo JSON
    objetos_json = [json.loads(line) for line in archivo]
# Convierte los datos en un DataFrame
df_categorias = pd.DataFrame(objetos_json)

In [58]:
df_categorias

,id_categoria,categoria,n_categoria
0,0,restaurant,restaurant
1,1,buffet restaurant,buffet restaurant
2,2,bakery,bakery
3,3,deli,deli
4,4,gift basket store,gift basket store
...,...,...,...
1610,1610,damage restoration,damage restoration
1611,1611,hair loss centers,hair loss centers
1612,1612,car window tinting,car window tinting
1613,1613,wigs,wig


In [59]:
# Realizamos una union de las tablas de categorias y de restaurantes por similitud en la categoria
df_categorias_restaurantes = pd.merge(df_restaurantes, df_categorias, left_on='categorias', right_on='categoria', how='left')

In [60]:
df_categorias_restaurantes

,index,nombre,identificador_google,categorias,identificador_yelp,id_categoria,categoria,n_categoria
0,0,Cape Seafood Shack,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,restaurant,None,0,restaurant,restaurant
1,1,Fresh Point Country Buffet,0x88c2e4e34f1ed783:0x76c5da381c499d79,buffet restaurant,None,1,buffet restaurant,buffet restaurant
2,2,Oneyda's Bakery,0x88dae191ee505917:0x6ba3e25388d3fad4,bakery,None,2,bakery,bakery
3,2,Oneyda's Bakery,0x88dae191ee505917:0x6ba3e25388d3fad4,deli,None,3,deli,deli
4,3,Annie's Bake Shoppe,0x88d9c754413f6c9d:0x1f93eff5e0ba9c16,bakery,None,2,bakery,bakery
...,...,...,...,...,...,...,...,...
95421,26610,Colony Grill - St. Petersburg,None,beer bar,esBGrrmuZzSiECyRBoKvvA,1066,beer bar,beer bar
95422,26610,Colony Grill - St. Petersburg,None,nightlife,esBGrrmuZzSiECyRBoKvvA,1062,nightlife,nightlife
95423,26610,Colony Grill - St. Petersburg,None,wine bars,esBGrrmuZzSiECyRBoKvvA,1131,wine bars,wine bars
95424,26610,Colony Grill - St. Petersburg,None,pizza,esBGrrmuZzSiECyRBoKvvA,1081,pizza,pizza


In [61]:
# Agrupamos por categoria, obteniendo una lista de los id de los restaurantes
df_categorias_restaurantes = df_categorias_restaurantes.groupby('n_categoria')['index'].agg(list).reset_index()

In [62]:
df_categorias_restaurantes

,n_categoria,index
0,aa shop,"[663, 926, 1465, 3029, 3702, 3915, 4466, 4791,..."
1,acai bowls,"[15354, 15763, 15966, 16220, 16273, 16336, 164..."
2,accessory,"[15298, 16487, 19007, 19201, 20014, 22985, 245..."
3,active life,"[15233, 15602, 15617, 15658, 15777, 15784, 158..."
4,acupuncture,[21597]
...,...,...
1476,yoga,"[15233, 17374, 20577, 23201, 24389]"
1477,yoga instructor,[3367]
1478,yoga retreat center,"[3367, 14178]"
1479,yoga studio,"[1852, 3367, 10802, 11330, 11427, 13143, 14178..."


In [63]:
# Expandimos los ids de los restaurantes, para obtener categorias y restaurantes 1 a 1
df_categorias_restaurantes = df_categorias_restaurantes.explode('index')
# Renombramos las columnas
df_categorias_restaurantes.columns= ['n_categoria', 'id_restaurant']
# Creamos un nuevo indice para nuestros datos
df_categorias_restaurantes = df_categorias_restaurantes.reset_index(drop=True).reset_index()

In [64]:
df_categorias_restaurantes

,index,n_categoria,id_restaurant
0,0,aa shop,663
1,1,aa shop,926
2,2,aa shop,1465
3,3,aa shop,3029
4,4,aa shop,3702
...,...,...,...
95421,95421,yoga studio,11427
95422,95422,yoga studio,13143
95423,95423,yoga studio,14178
95424,95424,yoga studio,14917


In [65]:
# Eliminamos los duplicados en la nueva categoria creada por nosotros
df_categorias = df_categorias.drop_duplicates(subset=["n_categoria"])
# Eliminamos las categorias originales
df_categorias = df_categorias.drop(['categoria'], axis=1)
# Renombramos
df_categorias.columns = ['id_categoria', 'categoria']

In [66]:
df_categorias

,id_categoria,categoria
0,0,restaurant
1,1,buffet restaurant
2,2,bakery
3,3,deli
4,4,gift basket store
...,...,...
1610,1610,damage restoration
1611,1611,hair loss centers
1612,1612,car window tinting
1613,1613,wig


In [67]:
# Unimos las tablas por nombre de categoria
df_categorias_restaurantes = pd.merge(df_categorias_restaurantes, df_categorias, left_on='n_categoria', right_on='categoria', how='left')
# Eliminamos las columnas que ya no serviran
df_categorias_restaurantes = df_categorias_restaurantes.drop(['n_categoria', 'categoria'], axis=1)
# Renombramos y ordenamos
df_categorias_restaurantes.columns = ['id_categoria_restaurante', 'id_restaurante', 'id_categoria']
df_categorias_restaurantes = df_categorias_restaurantes[['id_categoria_restaurante','id_categoria', 'id_restaurante']]

In [68]:
df_categorias_restaurantes

,id_categoria_restaurante,id_categoria,id_restaurante
0,0,249,663
1,1,249,926
2,2,249,1465
3,3,249,3029
4,4,249,3702
...,...,...,...
95421,95421,399,11427
95422,95422,399,13143
95423,95423,399,14178
95424,95424,399,14917


In [69]:
# Guardar el DataFrame como un archivo JSON
df_categorias.to_json('categorias.json', orient='records', lines=True)

# Guardar el DataFrame como un archivo JSON
df_categorias_restaurantes.to_json('categoriasrestaurantes.json', orient='records', lines=True)